# Red neuronal artificial

### Importación de las librerías

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
tf.__version__

'2.2.0'

## Parte 1 - Preprocesamiento de datos

### Importar el conjunto de datos

In [ ]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [ ]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [ ]:
print(y)

[1 0 1 ... 1 1 0]


### Codificación de datos categóricos

Etiqueta Codificación de la columna "Género"

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [ ]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Codificación de la columna "Geografía"

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### División del conjunto de datos en conjunto de entrenamiento y conjunto de prueba

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Escalado de características

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Parte 2 - Construcción de la RNA

### Inicialización de la RNA

In [ ]:
ann = tf.keras.models.Sequential()

### Añadir la capa de entrada y la primera capa oculta

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Añadir la segunda capa oculta

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Añadir la capa de salida

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Parte 3 - Entrenamiento de la RNA

### Compilación de la RNA

In [ ]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Entrenamiento de la RNA en el conjunto de entrenamiento

In [ ]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 0s 1ms/step - loss: 0.8037 - accuracy: 0.5185
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5291 - accuracy: 0.7901
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4888 - accuracy: 0.7952
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4668 - accuracy: 0.7979
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4478 - accuracy: 0.7994
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4302 - accuracy: 0.8049
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4123 - accuracy: 0.8119
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3947 - accuracy: 0.8238
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3807 - accuracy: 0.8355
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3720 - accura

## Parte 4 - Realización de predicciones y evaluación del modelo

### Predecir el resultado de una sola observación

**Tarea**

Utilice nuestro modelo RNA para predecir si el cliente con la siguiente información abandonará el banco:

Geografía: Francia

Puntuación crediticia: 600

Sexo: Hombre Hombre

Edad: 40 años

Tenencia: 3 años

Saldo: \$ 60000

Número de productos: 2

¿Este cliente tiene tarjeta de crédito? Sí

Es este cliente un Miembro Activo: Sí

Salario estimado: \$ 50000

Entonces, ¿debemos despedirnos de ese cliente?

**Solution**

In [ ]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

[[False]]


Por tanto, nuestro modelo RNA predice que este cliente se queda en el banco.

**Nota importante 1:** Observe que los valores de las características se introdujeron todos en un par doble de corchetes. Esto se debe a que el método "predict" siempre espera un array 2D como formato de entrada. Y poner nuestros valores en un par doble de corchetes hace que la entrada sea exactamente un array 2D.
**Nota importante 2:** Observe también que el país "Francia" no se introdujo como cadena en la última columna, sino como "1, 0, 0" en las tres primeras. Eso es porque, por supuesto, el método de predicción espera los valores codificados de un solo golpe del estado, y como vemos en la primera fila de la matriz de características X, "Francia" se codificó como "1, 0, 0". Y cuidado con incluir estos valores en las tres primeras columnas, porque las variables ficticias siempre se crean en las primeras columnas.

### Predicción de los resultados de la prueba

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1516   79]
 [ 200  205]]


0.8605